# Modelo 11 - Adaptado de Regressor a Clasificación

In [ ]:
# === Importación de librerías ===
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

# === Carga de datos ===
df_train = pd.read_csv("./data/train.csv")
df_test = pd.read_csv("./data/test.csv")

# === Eliminar columnas con más del 40% de nulos ===
null_ratio = df_train.isnull().mean()
cols_to_drop = null_ratio[null_ratio > 0.4].index.tolist()
df_train.drop(columns=cols_to_drop, inplace=True)
df_test.drop(columns=[col for col in cols_to_drop if col in df_test.columns], inplace=True)

# === Imputación de valores nulos ===
for df in [df_train, df_test]:
    for col in df.columns:
        if df[col].isnull().sum() > 0:
            if df[col].dtype in ['float64', 'int64']:
                df[col].fillna(df[col].mean(), inplace=True)
            else:
                df[col].fillna(df[col].mode()[0], inplace=True)

# === Codificación robusta solo para columnas comunes y categóricas ===
cat_cols = df_train.select_dtypes(include=['object', 'category']).columns
cat_cols = [col for col in cat_cols if col in df_test.columns and col != 'RENDIMIENTO_GLOBAL']
for col in cat_cols:
    le = LabelEncoder()
    df_train[col] = le.fit_transform(df_train[col])
    mapping = dict(zip(le.classes_, le.transform(le.classes_)))
    df_test[col] = df_test[col].map(mapping).fillna(-1).astype(int)

# === Variable objetivo ===
X = df_train.drop(columns=['RENDIMIENTO_GLOBAL'])
y = df_train['RENDIMIENTO_GLOBAL']

# === Identificar columna ID automáticamente ===
id_col = [col for col in df_test.columns if col.lower() == 'id']
if not id_col:
    raise KeyError("No se encontró una columna llamada 'Id' o 'ID' en df_test.")
id_col = id_col[0]
X_test_model = df_test.drop(columns=[id_col])

# === Entrenamiento ===
modelo = RandomForestClassifier(n_estimators=100, random_state=42)
modelo.fit(X, y)

# === Predicción ===
X_test_model = df_test
y_pred = modelo.predict(X_test_model)

# === Generar archivo de submission ===
submission = pd.DataFrame({'ID': df_test[id_col], 'RENDIMIENTO_GLOBAL': y_pred})
submission.to_csv('submission.csv', index=False)

C:\Users\pipey\AppData\Local\Temp\ipykernel_12212\2412213343.py:23: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mode()[0], inplace=True)


KeyboardInterrupt: 